In [2]:
import os
os.chdir('../')

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzipped_data_dir: Path

In [4]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories

In [5]:
@dataclass
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzipped_data_dir=config.unzipped_data_dir
        )
        
        return data_ingestion_config

In [9]:
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size
from pathlib import Path

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self)-> str:
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs(self.config.root_dir, exist_ok = True)
            logger.info(f"downloading data from {dataset_url} into file {zip_download_dir}")
            
            file_id = dataset_url.split('/')[-2]
            gdown.download(id=file_id, output=zip_download_dir)
            size = get_size(Path(zip_download_dir))
            logger.info(f"file downloaded successfully and saved at {zip_download_dir} with size {size}")
            return zip_download_dir
        except Exception as e:
            raise e
    
    
    def extract_zip_file(self, zip_file_path: str):
        try:
            logger.info(f"extraction of file: {zip_file_path} started")
            unzip_dir = self.config.unzipped_data_dir
            os.makedirs(unzip_dir, exist_ok = True)
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_dir)
            logger.info(f"extraction completed successfully at location: {unzip_dir}")
        except Exception as e:
            raise e
        
        

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    
    data_ingestion = DataIngestion(config = data_ingestion_config)
    zip_file_path = data_ingestion.download_file()
    data_ingestion.extract_zip_file(zip_file_path=zip_file_path)
except Exception as e:
    raise e

[2025-12-20 09:08:41,101: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-20 09:08:41,157: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-20 09:08:41,162: INFO: common: created directory at : artifacts]
[2025-12-20 09:08:41,166: INFO: common: created directory at : artifacts/data_ingestion]
[2025-12-20 09:08:41,171: INFO: 2883134549: downloading data from https://drive.google.com/file/d/1_wHDPchQbw4-huJUvVadvM9BW5y-KZ8u/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?id=1_wHDPchQbw4-huJUvVadvM9BW5y-KZ8u
From (redirected): https://drive.google.com/uc?id=1_wHDPchQbw4-huJUvVadvM9BW5y-KZ8u&confirm=t&uuid=7a7f4e80-1176-4d4e-8f1f-b12d8ebaccfb
To: f:\projects\project-9 kidney disease\kidney-disease-classification\artifacts\data_ingestion\data.zip
  0%|          | 4.72M/1.63G [00:04<20:02, 1.35MB/s]

KeyboardInterrupt: 

  0%|          | 4.72M/1.63G [00:20<20:02, 1.35MB/s]